In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel("ChatData.xlsx")

In [3]:
df.index = range(552) 

In [4]:
for i in range(552):
    df['Conversation'][i] = re.sub("\([0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}\)+", ' nr', df['Conversation'][i])
    

C:\Users\Sachit\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
for i in range(552):
    df['Visitor Name'][i] = df['Visitor Name'][i].lstrip(' ')

C:\Users\Sachit\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [6]:
Chats = pd.DataFrame(columns=['Conversation Num', 'Visitor', 'Chatbot'])
Chats

,Conversation Num,Visitor,Chatbot


In [7]:
Conv = Chats

In [8]:
Visitor = []
CustomerS = []
User = []

for i in range(552):
    p=0
    chatItems = re.split(' nr ' + df['Visitor Name'][i]+ ': | nr Customer | nr Ang', df['Conversation'][i])             
    for l in range(len(chatItems)):
        if p>0:
            p=p-1
            pass  
        else:
            j = chatItems[l]
            if j != '':
                if j[:4] == 'el: ':
                    CustomerS.append([j[4:], i])
                    for k in range(len(chatItems)-l-1):
                        if chatItems[l+1][:4] == 'el: ':
                            CustomerS[-1].append(chatItems[l+1][4:])
                            l=l+1
                            p=p+1
                        elif chatItems[l+1][:9] == 'Service: ':
                            CustomerS[-1].append(chatItems[l+1][9:])    
                            l=l+1
                            p=p+1
                        else:
                            break
                     
                        
                elif j[:9] == 'Service: ':
                    CustomerS.append([j[9:], i])
                    for k in range(len(chatItems)-l-1):
                        if chatItems[l+1][:4] == 'el: ':
                            CustomerS[-1].append(chatItems[l+1][4:])
                            l=l+1
                            p=p+1
                        elif chatItems[l+1][:9] == 'Service: ':
                            CustomerS[-1].append(chatItems[l+1][9:])
                            l=l+1
                            p=p+1
                        else:
                            break
                       
                  
                else:
                    User.append([j, i])
                    for k in range(len(chatItems)-l-1):
                        if chatItems[l+1][:4] != 'el: ' and chatItems[l+1][:9] != 'Service: ':
                            User[-1].append(chatItems[l+1])
                            l=l+1
                            p=p+1
                        else:
                            break
                       
    

In [9]:
data = Conv.append(User)

C:\Users\Sachit\Anaconda3\lib\site-packages\pandas\indexes\api.py:71: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)


In [10]:
data['Visitor'] = data[0].map(str) + '\n' +data[2].map(str)
for i in range(3,11):
    data['Visitor'] = data['Visitor'] + '\n' +data[i].map(str)

In [11]:
for i in range(len(data)):
    data['Visitor'][i] = data['Visitor'][i].replace('None', '')
    data['Visitor'][i] = data['Visitor'][i].rstrip('\n')

C:\Users\Sachit\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Sachit\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [12]:
data['Conversation Num'] = data[1]

In [13]:
data = data[['Conversation Num', 'Visitor', 'Chatbot']]

In [14]:
pd.options.display.float_format = '{:,.0f}'.format

In [15]:
k = 0
for i in range(len(data)):
    if k < len(CustomerS):
        if data['Conversation Num'][i] == CustomerS[k][1]:
            CustomerS[k].remove(CustomerS[k][1])
            data['Chatbot'][i] = '\n'.join(CustomerS[k])
            k = k+1

C:\Users\Sachit\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
data.head()

,Conversation Num,Visitor,Chatbot
0,0,wat about job?,We provide complete assistance for the placements
1,0,it means no guarantee?,NaN
2,1,"BE IT, MBA Marketing\nPune\nmore than 3+ years...",We apologize for keeping you waiting. Our oper...
3,1,paresh\nwaht is the total fees of digital mark...,"Fees For DMM is INR 7,45,000\nhttp://www.spjai..."
4,2,I want to know the prospects SP Jain Global MB...,We apologize for keeping you waiting. Our oper...


In [17]:
pd.isnull(data).sum()

Conversation Num      0
Visitor               0
Chatbot             211
dtype: int64

In [19]:
writer = pd.ExcelWriter(path='C:/Users/Sachit/Desktop/BDAP/ChatBot/GT/Messages.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
data.to_excel(writer, sheet_name='Messages')
writer.save()